# Watchlist example notebook

`Author: Francisco Förster, Last update: 20210411`

Here we will do a xmatch against a list of positions in the sky, given in the file ```watchlist.csv```

In [2]:
import pandas as pd
import psycopg2
import requests

# Get credentials to open a direct connection with the database

In [3]:
credentials_file = "https://raw.githubusercontent.com/alercebroker/usecases/master/alercereaduser_v4.json"
params = requests.get(credentials_file).json()["params"]

In [4]:
conn = psycopg2.connect(dbname=params["dbname"], user=params["user"], host=params["host"], password=params["password"])

# Get watchlist of object positions

In [6]:
df = pd.read_csv("https://github.com/alercebroker/usecases/blob/master/example_data/watchlist.csv?raw=True")
df.head()

,id_source,ra,dec
0,source_1,160.183014,33.016467
1,source_2,174.215249,44.837895


# Prepare the query string for watchlist for object within 1 degress of given positions, and  happening within the last 10 days

In [7]:
objects = []
for _,row in df.iterrows():
    objects.append(f"(\'{row.id_source}\', {row.ra}, {row.dec})")
objects_str = ",\n".join(objects)
objects_str

"('source_1', 160.18301441363647, 33.0164673528409),\n('source_2', 174.21524897555543, 44.83789535222221)"

Create query string

In [8]:
from astropy.time import Time
nt = Time.now()
last_mjd_discovery = nt.mjd - 10

In [9]:
search_radius = 1 # degrees
query = """
WITH catalog ( source_id, ra, dec) AS (
    VALUES
        {values}
)
SELECT 
    c.source_id, c.ra, c.dec, o.oid, o.meanra, o.meandec, q3c_dist(c.ra,c.dec,o.meanra,o.meandec), 
    o.firstmjd

FROM object o, catalog c
    /*
     * It is REALLY important to first use the catalog then the object ra,dec for speed. The radius is in degrees.
     */
WHERE
    q3c_join(c.ra, c.dec,o.meanra, o.meandec, {radius})
    AND o.firstmjd > %s
""" % (last_mjd_discovery)
query_str = query.format(values = objects_str, radius = search_radius) # radius in degrees

In [10]:
matches = pd.read_sql(query_str,conn)
print(matches.shape)
matches

(235, 8)


,source_id,ra,dec,oid,meanra,meandec,q3c_dist,firstmjd
0,source_1,160.183014,33.016467,ZTF21aarnexs,160.062296,33.670591,0.661852,59306.251771
1,source_1,160.183014,33.016467,ZTF21aarnfde,159.650987,32.392743,0.767755,59306.251771
2,source_1,160.183014,33.016467,ZTF21aarnfdl,160.053760,33.204848,0.217276,59306.251771
3,source_1,160.183014,33.016467,ZTF21aarnfdp,160.960105,33.015154,0.651606,59306.251771
4,source_1,160.183014,33.016467,ZTF21aarnfdj,159.557127,32.924597,0.533065,59306.251771
...,...,...,...,...,...,...,...,...
230,source_2,174.215249,44.837895,ZTF21aauhmmw,174.132976,44.515377,0.327782,59315.238819
231,source_2,174.215249,44.837895,ZTF21aauhmrg,173.903485,45.697832,0.887486,59315.238819
232,source_2,174.215249,44.837895,ZTF21aauhprj,174.667906,44.081964,0.822074,59315.239282
233,source_2,174.215249,44.837895,ZTF21aauhpri,174.260736,44.197326,0.641390,59315.239282


# Query objects within 10 degress of given positions, first detected within the last 10 days, with SN probabilities > 0.4, and ranking=1 in the stamp classifier

In [11]:
search_radius = 10 # degrees
query = """
WITH catalog ( source_id, ra, dec) AS (
    VALUES
        {values}
),
sn (oid, classifier_name, class_name, probability, ranking) AS (
    SELECT
        o.oid, p.classifier_name, p.class_name, p.probability, p.ranking
    FROM
        probability p
    INNER JOIN 
        object o
    ON 
        o.oid=p.oid
    WHERE
        p.classifier_name='stamp_classifier'
        AND p.class_name IN ('SN')
        AND p.ranking=1
        AND p.probability > 0.4
        AND o.firstmjd > {last_mjd_discovery}
)

SELECT 
    c.source_id, c.ra, c.dec, o.oid, o.meanra, o.meandec, q3c_dist(c.ra,c.dec,o.meanra,o.meandec), 
    o.firstmjd, sn.classifier_name, sn.class_name, sn.probability, sn.ranking

FROM object o INNER JOIN sn ON sn.oid=o.oid, catalog c
    /*
     * It is REALLY important to first use the catalog then the object ra,dec for speed. The radius is in degrees.
     */
WHERE
    q3c_join(c.ra, c.dec,o.meanra, o.meandec, {radius})
    AND o.firstmjd > {last_mjd_discovery}
ORDER BY
    c.source_id
"""
query_str = query.format(values = objects_str, radius = search_radius, last_mjd_discovery=last_mjd_discovery) # radius in degrees

In [12]:
matches = pd.read_sql(query_str,conn)
print(matches.shape)
matches.head(100)

(23, 12)


,source_id,ra,dec,oid,meanra,meandec,q3c_dist,firstmjd,classifier_name,class_name,probability,ranking
0,source_1,160.183014,33.016467,ZTF21aasvhwh,166.770659,32.800088,5.533925,59306.372836,stamp_classifier,SN,0.707814,1
1,source_1,160.183014,33.016467,ZTF21aasuggn,166.212813,31.622934,5.281848,59308.227685,stamp_classifier,SN,0.430196,1
2,source_1,160.183014,33.016467,ZTF21aastzsn,148.786835,35.379685,9.711189,59310.249132,stamp_classifier,SN,0.793107,1
3,source_1,160.183014,33.016467,ZTF21aappwvm,165.757943,31.974741,4.815426,59308.227685,stamp_classifier,SN,0.661527,1
4,source_1,160.183014,33.016467,ZTF21aatuynj,154.108685,24.817833,9.766739,59312.234653,stamp_classifier,SN,0.816249,1
5,source_1,160.183014,33.016467,ZTF21aasuhnu,160.418486,26.539921,6.479763,59308.269213,stamp_classifier,SN,0.754478,1
6,source_1,160.183014,33.016467,ZTF21aalwdzx,151.070306,38.973164,9.469663,59314.267894,stamp_classifier,SN,0.412541,1
7,source_1,160.183014,33.016467,ZTF21aatxuyp,157.572440,37.975224,5.394375,59314.277234,stamp_classifier,SN,0.771551,1
8,source_1,160.183014,33.016467,ZTF21aardzlu,155.949496,42.275894,9.843950,59306.248542,stamp_classifier,SN,0.651708,1
9,source_1,160.183014,33.016467,ZTF21aaroydq,163.573451,34.238847,3.076112,59306.304271,stamp_classifier,SN,0.524896,1


# Open the resulting objects in the ALeRCE explorer

In [13]:
url = "https://alerce.online/?" + "&".join(["oid=%s" % oid for oid in matches.oid.values]) + "&count=true&page=1&perPage=1000&sortDesc=false"
print(url)

https://alerce.online/?oid=ZTF21aasvhwh&oid=ZTF21aasuggn&oid=ZTF21aastzsn&oid=ZTF21aappwvm&oid=ZTF21aatuynj&oid=ZTF21aasuhnu&oid=ZTF21aalwdzx&oid=ZTF21aatxuyp&oid=ZTF21aardzlu&oid=ZTF21aaroydq&oid=ZTF21aardzpu&oid=ZTF21aarowtc&oid=ZTF21aarnbox&oid=ZTF21aasgnqa&oid=ZTF21aarohyu&oid=ZTF21aarwlht&oid=ZTF21aatwunv&oid=ZTF21aarxsfm&oid=ZTF21aarovmb&oid=ZTF21aardszu&oid=ZTF21aasyydo&oid=ZTF21aaszaoy&oid=ZTF21aarxuih&count=true&page=1&perPage=1000&sortDesc=false
